In [2]:
import requests
import bs4
import pandas as pd
import time

In [3]:
price_url = 'https://fchart.stock.naver.com/sise.nhn?symbol=005930&timeframe=day&count=1500&requestType=0'
price_data = requests.get(price_url)

In [4]:
price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml') # price_data.text를 lxml 옵션으로 bs에알려줌

In [5]:
item_list = price_data_bs.find_all('item') #item 태그로 에 해당하는 데이터를 리스트 형태로 저장

In [18]:
temp = item_list[0]['data'] #item_list의 첫번째 데이터를 temp로 지정
temp.split('|') # | 를 기준으로 temp를 나눠서 리스트 형태로 반환

['20140923', '23560', '23700', '23220', '23220', '390828']

In [19]:
for item in item_list: 
    temp_data = item['data']
    datas = temp_data.split('|')  #for문 돌면서 데이터 가공
    print(datas[0], datas[4])  

20140923 23220
20140924 23000
20140925 23120
20140926 23700
20140929 23900
20140930 23680
20141001 23120
20141002 22820
20141006 23020
20141007 23240
20141008 22600
20141010 22100
20141013 22160
20141014 22540
20141015 22500
20141016 22300
20141017 21780
20141020 22120
20141021 21660
20141022 22080
20141023 21880
20141024 22240
20141027 21900
20141028 21820
20141029 22600
20141030 23620
20141031 24880
20141103 24700
20141104 24340
20141105 24040
20141106 24080
20141107 24120
20141110 25360
20141111 24620
20141112 24600
20141113 24000
20141114 23880
20141117 24100
20141118 24400
20141119 24360
20141120 24200
20141121 24460
20141124 24400
20141125 23800
20141126 24020
20141127 25280
20141128 25740
20141201 25900
20141202 25900
20141203 25960
20141204 26000
20141205 26040
20141208 26380
20141209 26980
20141210 26340
20141211 26100
20141212 25720
20141215 25460
20141216 25580
20141217 25320
20141218 25320
20141219 26560
20141222 26900
20141223 26780
20141224 26860
20141226 27040
20141229 2

In [20]:
date_list = [] #date_list 빈 리스트 생성
price_list = [] #price_list 빈 리스트 생성
for item in item_list:  
    temp_data = item['data']
    datas = temp_data.split('|') #for문 돌면서 데이터 가공
    date_list.append(datas[0]) # data_list 리스트에 [0] 적용
    price_list.append(datas[4]) #price_list 리스트에 [4] 적용

price_df = pd.DataFrame({'종가':price_list}, index=date_list) #판다스를 이용해 index = data_list, 종가 = price_list 지정

In [17]:
price_df

,종가
20140923,23220
20140924,23000
20140925,23120
20140926,23700
20140929,23900
...,...
20201023,60200
20201026,60400
20201027,59800
20201028,59000


In [23]:
def make_price_dataframe(code, timeframe, count): #가격데이터 함수지정 종목코드, 시간, 갯수
    url = 'https://fchart.stock.naver.com/sise.nhn?requestType=0'
    price_url = url + '&symbol=' + code + '&timeframe=' + timeframe + '&count=' + count #주소 가공

    price_data = requests.get(price_url) #주소 지정
    price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml') #bs를 이용해 데이터 불러옴
    item_list = price_data_bs.find_all('item') #item태그 다찾아서 item_list 에 저장
    date_list =[] #날짜 리스트 생성
    price_list = [] #가격 리스트 생성
    for item in item_list: 
        temp_data = item['data']
        datas = temp_data.split('|') #for문 돌면서 가공
        date_list.append(datas[0]) #날짜 리스트에 저장
        price_list.append(datas[4]) #가격 리스트에 저장
      
    price_df = pd.DataFrame({code : price_list}, index=date_list) #날짜(인덱스) 가격 저장

    return price_df

In [26]:
make_price_dataframe('005930', 'day', '10')

,005930
20201014,60900
20201015,60000
20201016,59500
20201019,60000
20201020,60900
20201021,60900
20201022,60100
20201023,60200
20201026,60400
20201027,59800


In [28]:
firmcode_list = ['005930', '005380', '035420', '003550', '034730']

for num, code in enumerate(firmcode_list): #for문 생성
    price_df = make_price_dataframe(code, 'day', '1500') 
    if num == 0:
        total_price = price_df
    else:
        total_price = pd.merge(total_price, price_df, how='outer', right_index=True, left_index=True) # merge옆으로 병합

In [29]:
total_price

,005930,005380,035420,003550,034730
20140919,24200,195000,158625,71600,220500
20140922,23760,195500,158224,73400,225500
20140923,23220,191500,167037,75800,232500
20140924,23000,192000,166837,75800,238500
20140925,23120,189500,161830,75700,245500
...,...,...,...,...,...
20201021,60900,165500,297500,72400,201000
20201022,60100,162500,293000,72500,199500
20201023,60200,167000,286500,72200,202000
20201026,60400,171500,281500,71600,196500


In [31]:
path = r'C:\Users\gjals\Desktop\코딩\data.xls' #엑셀파일 지정
code_data = pd.read_excel(path) #판다스로 읽어옴
code_data = code_data[['종목코드', '기업명']]

def make_code2(x):
    x = str(x)
    return '0' * (6-len(x)) + x #종목코드 가공

code_data['종목코드'] = code_data['종목코드'].apply(make_code2)  #모든데이터에 적용

In [34]:
for num, code in enumerate(code_data['종목코드']): #종목코드로 도는 for문 생성
    try: #시도(1)
        print(num, code) #num, code 출력
        time.sleep(1) #1초쉬고
        try: #시도(2)
            price_df = make_price_dataframe(code, 'day', '1500') #make_price_dataframe 함수 실행
        except requests.exception.Timeout: #시도(2) 에서 타임아웃 나오면
            time.sleep(60) #60초 멈춤
            price_df = make_price_dataframe(code, 'day', '1500') #다시 함수 실행
        if num == 0: 
            total_price = price_df #첫번째 행 지정
        else:
            total_price = pd.merge(total_price, price_df, how='outer', right_index=True, left_index=True) #첫행부터 막행까지 옆으로 병합
    except ValueError: #시도(1) valueerror시
        continue #계속
    except KeyError: #시도(1) keyerror시
        continue #계속

0 060310
1 095570
2 006840
3 054620
4 265520
5 211270
6 027410
7 282330
8 138930
9 001460
10 001040
11 079160
12 035760
13 000120
14 011150
15 097950
16 051500
17 058820
18 023460
19 065770
20 083660
21 000590
22 012030
23 322780
24 016610
25 005830
26 000990
27 139130
28 068790
29 004840
30 241520
31 155660
32 069730
33 017940
34 245620
35 037370
36 079190
37 007700
38 130500
39 114090
40 900290
41 078930
42 083450
43 006360
44 001250
45 007070
46 028150
47 045890
48 078150
49 012630
50 039570
51 089470
52 294870
53 011200
54 036640
55 082740
56 299170
57 335890
58 335870
59 351340
60 351320
61 095340
62 099520
63 175330
64 950170
65 234080
66 067290
67 001060
68 096760
69 035900
70 024840
71 105560
72 024120
73 021320
74 036670
75 009440
76 112190
77 119650
78 044180
79 092220
80 151860
81 016380
82 046440
83 035600
84 001390
85 060720
86 001940
87 122450
88 052900
89 058400
90 114450
91 025000
92 092230
93 000040
94 044450
95 030210
96 036030
97 058850
98 058860
99 060370
100 093050

In [35]:
total_price.index = pd.to_datetime(total_price.index) #날짜를 날짜형 인덱스로 바꿔줌 ex)20201029 -> 2020-10-29
total_price.to_excel(r'C:\Users\gjals\Desktop\코딩\가격데이터.xlsx') #엑셀로 저장